In [ ]:
from glob import glob
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from skimage import data, io
from skimage.color import label2rgb
from skimage.filters import sobel
from skimage.measure import label
from skimage.segmentation import expand_labels, watershed
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import segmentation
from skimage import data, restoration, util
from skimage import data, morphology
from skimage.filters import threshold_otsu, threshold_yen
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, area_closing
from skimage.color import label2rgb
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
DATASET_ROOT = r"./pollen_dataset/"
OBJECTS_ROOT_1 = r"./pollen_dataset_stage_1/"

In [ ]:
all_images = glob(os.path.join(DATASET_ROOT, "**", "*.jpg"), recursive=True)
print(f"Total amount of images: {len(all_images)}")

# Visualisation 

In [ ]:
def draw_detections(image, label_image):
    # to make the background transparent, pass the value of `bg_label`,
    # and leave `bg_color` as `None` and `kind` as `overlay`
    image_label_overlay = label2rgb(label_image, image=image, bg_label=0)
    
    fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(6, 3))
    ax1.imshow(image)
    ax2.imshow(image_label_overlay)
    # for region in regionprops(label_image):
    #     # draw rectangle around segmented coins
    #     minr, minc, maxr, maxc = region.bbox
    #     rect = mpatches.Rectangle(
    #         (minc, minr),
    #         maxc - minc,
    #         maxr - minr,
    #         fill=False,
    #         edgecolor='red',
    #         linewidth=2,
    #     )
    #     ax.add_patch(rect)
    
    ax1.set_axis_off()
    ax2.set_axis_off()
    plt.tight_layout()
    plt.show()

# Detect objects

In [ ]:
def object_detection_stage(image_rgb, radius=50):
    image_rgb = np.copy(image_rgb)
    image = rgb2gray(image_rgb)
    # apply threshold
    thresh = threshold_yen(image)
    bw = closing(image < thresh, square(25))
    bw =  morphology.remove_small_holes(
        morphology.remove_small_objects(bw, 5000), 5000
    )
    bw = morphology.isotropic_erosion(bw, radius)
    bw = morphology.isotropic_dilation(bw, radius)
    
    # # remove artifacts connected to image border
    # cleared = clear_border(bw)
    # label image regions
    label_image = label(bw)
    return label_image

In [ ]:
image_rgb = io.imread(all_images[2])
label_image = object_detection_stage(image_rgb)
draw_detections(image_rgb, label_image)

# Extract detected objects

# Separate Objects

In [ ]:
def extract_objects(image_rgb, labels_image, min_distance, padding):
    labels = ndi.binary_fill_holes(labels_image)
    labels = morphology.binary_erosion(labels)
    labels = morphology.binary_dilation(labels).astype(np.uint8)
    
    # # Now we want to separate the two objects in image
    # # Generate the markers as local maxima of the distance to the background
    distance = ndi.distance_transform_edt(labels)
    coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=labels, min_distance=min_distance)
    if len(coords) > 0:
        mask = np.zeros(distance.shape, dtype=bool)
        mask[tuple(coords.T)] = True
        draw_detections(distance, distance)
        markers, _ = ndi.label(mask)
        labels = watershed(-distance, markers, mask=labels)

    for region in regionprops(labels):
        # print(image_rgb_patch.shape)
        # print(region.bbox)
        minr, minc, maxr, maxc = region.bbox    
        # print(minr, minc, maxr, maxc)
        minr, minc, maxr, maxc = max(0, minr-padding), max(0, minc-padding), min(image_rgb.shape[0]-1, maxr+padding), min(image_rgb.shape[1]-1, maxc+padding)
        # print(minr, minc, maxr, maxc)
        label_patch = np.copy(labels[minr:maxr, minc:maxc])
        label_patch = clear_border(label_patch)
        image_rgb_patch = np.copy(image_rgb[minr:maxr, minc:maxc])
        image_rgb_patch = np.where((label_patch > 0)[...,None], image_rgb_patch, 0).astype(np.uint8)
        draw_detections(image_rgb_patch, label_patch)
    # candidates = []
    

def object_separation_stage(image_rgb, labels, area_threshold = 20, padding=10, min_distance=100):
    image_rgb = np.copy(image_rgb)
    for region in regionprops(labels):
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox            
        minr, minc, maxr, maxc = max(0, minr-padding), max(0, minc-padding), min(image_rgb.shape[0]-1, maxr+padding), min(image_rgb.shape[1]-1, maxc+padding)
        
        label_patch = np.copy(labels[minr:maxr, minc:maxc])
        label_patch = clear_border(label_patch)
        image_rgb_patch = np.copy(image_rgb[minr:maxr, minc:maxc])
        image_rgb_patch = np.where((label_patch > 0)[...,None], image_rgb_patch, 0).astype(np.uint8)
        extract_objects(image_rgb_patch, label_patch, min_distance, padding)

image_rgb = io.imread(all_images[2])
label_image = object_detection_stage(image_rgb)
object_separation_stage(image_rgb, label_image)

In [ ]:
for idx, image_path in tqdm(enumerate(all_images)):
    if idx % 20 == 0:
        image_rgb = io.imread(image_path)
        label_image = object_detection_stage(image_rgb)
        object_separation_stage(image_rgb, label_image)
    

In [ ]:
for imx, path in enumerate(tqdm(all_images)):
    rel_path = os.path.relpath(path, start=DATASET_ROOT)
    patch_template = os.path.splitext(rel_path)[0] + "_{}.jpg"
    patch_template = os.path.join(OBJECTS_ROOT_1, patch_template)
    os.makedirs(os.path.dirname(patch_template), exist_ok=True)   
    # for i, patch in enumerate(object_detection_stage_1(io.imread(path))):
    #     io.imsave(patch_template.format(i), patch)
    # if (imx>5):
    #     break